Import packages

In [1]:
from gurobipy import *
import numpy as np
import pandas as pd
import os
import re

In [2]:
os.chdir("/Users/ruhimahendra/Desktop/MMA/Decision Analytics/finalproj")

Import data from CSV files

In [3]:
raw_data = pd.read_excel('FLIP.xlsx')
travel_modes = ['Driving', 'Walking', 'Transit (Subway)', 'Biking']
driving_data = pd.read_csv('Origin Destination Matrix_driving2.csv', encoding='latin-1')
walking_data = pd.read_csv('Origin Destination Matrix_walking2.csv', encoding='latin-1')
transit_data = pd.read_csv('Origin Destination Matrix_transit&transit_mode=subway2.csv', encoding='latin-1')
biking_data = pd.read_csv('Origin Destination Matrix_bicycling2.csv', encoding='latin-1')

Create Dataframe to keep track of store names

In [4]:
stores = pd.DataFrame(['Loblaws MLG', 'No Frills', 'No Frills Joe\'s',
                       'Walmart','Grocery Gateway', 
                       'Loblaws', 'Metro'], 
                      index = [22,23,24,26,27,28,29], 
                      columns = ['Store_Name'])

Data Preprocessing

In [5]:
items = raw_data
#drop futile columns 
items = items.drop(columns = ['Unnamed: 1', 'FREESUG'])

#drop items with prices equal zero 
null_prices = items[items['Regular Price'] == 0.0].index
items = items.drop(null_prices)

#Sort items by store code and then ID ascendingly 
items = items.sort_values(by = ['Store Code', 'ID'])

#Drop items with all NFT missing 
items = items.drop(items[items['KCAL'].isna() & items['FAT'].isna() &  items['SATFAT'].isna() &  items['TRANS'].isna()&  items['CHOL'].isna() &  items['NA'].isna() &  items['CHO'].isna() &  items['FIBRE'].isna() &  items['SUGAR'].isna() &  items['PRO'].isna() &  items['K'].isna() &  items['VITA'].isna() &  items['VITC'].isna() &  items['CALCIUM'].isna()&  items['IRON'].isna()].index)

#Drop items with KCAL missing
items = items.drop(items[items['KCAL'].isna()].index)

#Drop items with price missing
items = items.drop(items[items['Regular Price'].isna()].index)

items = items.reset_index(drop = True)

I = len(items)

In [6]:
%%capture

#convert some container size units to grams and ml
strs = ['egg', r'\bL\b', 'gm', 'gr', 'lb']
for x in strs:
    for i in range(I):
        if bool(re.findall("\d+\.\d+", str(items['Container Size (as indicated on package)'][i]))):
            q = round(float(re.findall("\d+\.\d+", str(items['Container Size (as indicated on package)'][i]))[0]),2)
        elif bool(re.findall("\d+", str(items['Container Size (as indicated on package)'][i]))):
            q = round(float(re.findall("\d+", str(items['Container Size (as indicated on package)'][i]))[0]),2)
        if x == r'\bL\b':
            if x in str(items['Container Size (as indicated on package)'][i]):
                items['Container Size (ml)'][i] = 1000*q            
        elif x.casefold() in str(items['Container Size (as indicated on package)'][i]).casefold():
            if x == 'egg':
                items['Container Size (g)'][i] = 50*q
            elif x == 'kg':
                items['Container Size (g)'][i] = 1000*q
            elif x in ['gr', 'gm']:
                items['Container Size (g)'][i] = q
            elif x == 'lb':
                items['Container Size (g)'][i] = 453.592*q
                
#Drop all remaining items with container size other than grams and ml
items = items.drop(items[items['Container Size (g)'].isna() & items['Container Size (ml)'].isna()].index)

#Drop as indicated on package column
items = items.drop(columns = 'Container Size (as indicated on package)')
items = items.reset_index(drop = True)

I = len(items)

In [7]:
#Drop items with big container sizes in grams
items = items.drop(items[items['Container Size (g)'] > 1000].index)
items = items.reset_index(drop = True)

I = len(items)

In [8]:
# Getting number of servings in each container only for servings expressed in g or ml.
# Only serving sizes greater or equal to 1 and those less than the item container size are kept.
I = len(items)
servings = []
indices = []
for i in range(I):
    if (not bool(re.findall(r'\bg\b', items['Serving Size on NFT (other than grams)'][i].casefold()))) & (not bool(re.findall(r'\bml\b', items['Serving Size on NFT (other than grams)'][i].casefold()))):
        indices.append(i)
    else:
        if (bool(re.findall(r"\d+\.\d+ g\b", str(items['Serving Size on NFT (other than grams)'][i].casefold())))):
            Q = re.findall(r"\d+\.\d+ g\b", str(items['Serving Size on NFT (other than grams)'][i].casefold()))[0]
            q = float(re.findall(r"\d+\.\d+", Q)[0])
        elif bool(re.findall(r"\d+ g\b", str(items['Serving Size on NFT (other than grams)'][i].casefold()))):
            Q = re.findall(r"\d+ g\b", str(items['Serving Size on NFT (other than grams)'][i].casefold()))[0]
            q = float(re.findall(r"\d+", Q)[0])
            
        elif (bool(re.findall(r"\d+\.\d+g\b", str(items['Serving Size on NFT (other than grams)'][i].casefold())))):
            Q = re.findall(r"\d+\.\d+g\b", str(items['Serving Size on NFT (other than grams)'][i].casefold()))[0]
            q = float(re.findall(r"\d+\.\d+", Q)[0])
        elif bool(re.findall(r"\d+g\b", str(items['Serving Size on NFT (other than grams)'][i].casefold()))):
            Q = re.findall(r"\d+g\b", str(items['Serving Size on NFT (other than grams)'][i].casefold()))[0]
            q = float(re.findall(r"\d+", Q)[0])
                      
        elif (bool(re.findall(r"\d+\.\d+ ml\b", str(items['Serving Size on NFT (other than grams)'][i].casefold())))):
            Q = re.findall(r"\d+\.\d+ ml\b", str(items['Serving Size on NFT (other than grams)'][i].casefold()))[0]
            q = float(re.findall(r"\d+\.\d+", Q)[0])
        elif bool(re.findall(r"\d+ ml\b", str(items['Serving Size on NFT (other than grams)'][i].casefold()))):
            Q = re.findall(r"\d+ ml\b", str(items['Serving Size on NFT (other than grams)'][i].casefold()))[0]
            q = float(re.findall(r"\d+", Q)[0])
        
        elif (bool(re.findall(r"\d+\.\d+ml\b", str(items['Serving Size on NFT (other than grams)'][i].casefold())))):
            Q = re.findall(r"\d+\.\d+ml\b", str(items['Serving Size on NFT (other than grams)'][i].casefold()))[0]
            q = float(re.findall(r"\d+\.\d+", Q)[0])
        elif bool(re.findall(r"\d+ml\b", str(items['Serving Size on NFT (other than grams)'][i].casefold()))):
            Q = re.findall(r"\d+ml\b", str(items['Serving Size on NFT (other than grams)'][i].casefold()))[0]
            q = float(re.findall(r"\d+", Q)[0])
    
        if str(items['Container Size (g)'][i]).casefold() == 'nan':
            if (q > items['Container Size (ml)'][i]) | (q < 1):
                indices.append(i)
            else:
                servings.append(items['Container Size (ml)'][i]/q)
        else:
            if (q > items['Container Size (g)'][i]) | (q < 1):
                indices.append(i)
            else:
                servings.append(items['Container Size (g)'][i]/q)

items = items.drop(indices)
items = items.reset_index(drop = True)
items['Servings'] = servings
I = len(items)

In [9]:
# Some items have both grams and ml in their serving size
c = 0
I = len(items)
for i in range(I):
    if (bool(re.findall(r'\bg\b', items['Serving Size on NFT (other than grams)'][i].casefold()))) & (bool(re.findall(r'\bml\b', items['Serving Size on NFT (other than grams)'][i].casefold()))):
        c += 1


In [10]:
# 24 products have both g and ml in their serving sizes. Drop items
indices = []
for i in range(I):
    if ('g' in items['Serving Size on NFT (other than grams)'][i].casefold()) & ('ml' in items['Serving Size on NFT (other than grams)'][i].casefold()):
        indices.append(i)
        
items = items.drop(indices)
items = items.reset_index(drop = True)
I = len(items)

In [17]:
%%capture
#Filling all NA values in all NFTs with zero
I = len(items)
NFT = ['FAT', 'SATFAT', 'TRANS', 'CHOL', 'NA', 'CHO', 'FIBRE', 'SUGAR', 'PRO', 'K', 'VITA', 'VITC', 'CALCIUM', 'IRON']
for n in NFT:
    for i in items[items[n].isna()].index:
        items[n][i] = 0.0

In [12]:
# Convert the TRA categories into dummy variables
data = items
dummies = pd.get_dummies(data['TRA_Cat']).rename(columns=lambda x: 'Cat_' + str(x))
data= pd.concat([data, dummies], axis=1)

In [13]:
#Find the codes of stores left in data
store_code = np.unique(data['Store Code'])


# Updating stores dataframe to match remaining stores
stores = stores.loc[list(store_code),:]

In [14]:
# Convert final data into dictionary

grocery_data = {}                                                               

for i in store_code:
    d = {}    
    df = data[data["Store Code"]==i]
    distance_driving = driving_data[driving_data["store"] == i]
    distance_walking = walking_data[walking_data["store"] == i]
    distance_transit = transit_data[transit_data["store"] == i]
    distance_biking = biking_data[biking_data["store"] == i]
    d.update({'distance_driving': int(distance_driving["Travel Distance (m)"])})
    d.update({'duration_driving': int(distance_driving["Travel Duration (s)"])})
    d.update({'distance_walking': int(distance_walking["Travel Distance (m)"])})
    d.update({'duration_walking': int(distance_walking["Travel Duration (s)"])})
    d.update({'distance_transit': int(distance_transit["Travel Distance (m)"])})
    d.update({'duration_transit': int(distance_transit["Travel Duration (s)"])})
    d.update({'distance_biking': int(distance_biking["Travel Distance (m)"])})
    d.update({'duration_biking': int(distance_biking["Travel Duration (s)"])})
    d.update({'names':  df["Product Name"].tolist()})
    d.update({'prices':  df["Regular Price"].tolist()})
    d.update({'servings':  df["Servings"].tolist()})
    d.update({'calories': df["KCAL"].tolist()})
    d.update({'fat': df["FAT"].tolist()})
    d.update({'satfat': df["SATFAT"].tolist()})
    d.update({'trans': df["TRANS"].tolist()})  
    d.update({'chol': df["CHOL"].tolist()})  
    d.update({'na': df["NA"].tolist()})  
    d.update({'cho': df["CHO"].tolist()})  
    d.update({'fibre': df["FIBRE"].tolist()})
    d.update({'sugar': df["SUGAR"].tolist()}) 
    d.update({'pro': df["PRO"].tolist()})  
    d.update({'k': df["K"].tolist()})    
    d.update({'vita': df["VITA"].tolist()})
    d.update({'vitc': df["VITC"].tolist()})
    d.update({'calcium': df["CALCIUM"].tolist()})   
    d.update({'iron': df["IRON"].tolist()})  
    d.update({'ingredients': df["Ingredients"].tolist()}) 
    d.update({'A': df["Cat_A"].tolist()})  
    d.update({'B': df["Cat_B"].tolist()}) 
    d.update({'C': df["Cat_C"].tolist()}) 
    d.update({'D': df["Cat_D"].tolist()}) 
    d.update({'E': df["Cat_E"].tolist()}) 
    d.update({'F': df["Cat_F"].tolist()}) 
    d.update({'G': df["Cat_G"].tolist()}) 
    d.update({'H': df["Cat_H"].tolist()}) 
    d.update({'I': df["Cat_I"].tolist()}) 
    d.update({'J': df["Cat_J"].tolist()}) 
    d.update({'K': df["Cat_K"].tolist()}) 
    d.update({'L': df["Cat_L"].tolist()}) 
    d.update({'M': df["Cat_M"].tolist()}) 
    d.update({'N': df["Cat_N"].tolist()}) 
    d.update({'O': df["Cat_O"].tolist()}) 
    d.update({'P': df["Cat_P"].tolist()}) 
    d.update({'Q': df["Cat_Q"].tolist()}) 
    d.update({'R': df["Cat_R"].tolist()}) 
    d.update({'S': df["Cat_S"].tolist()}) 
    d.update({'T': df["Cat_T"].tolist()}) 
    d.update({'U': df["Cat_U"].tolist()}) 
    d.update({'V': df["Cat_V"].tolist()}) 
    d.update({'W': df["Cat_W"].tolist()}) 
    d.update({'X': df["Cat_X"].tolist()}) 
    d.update({'ZX': df["Cat_ZX"].tolist()}) 
    d.update({'ZY': df["Cat_ZY"].tolist()}) 
    grocery_data[i] = d


In [15]:
# Create indices for variables
indices = []
for i in grocery_data:
    for n in range(len(grocery_data[i]['names'])):
        I = (i,n)
        indices.append(I)

Create Decision variables for the model

In [18]:
prob = Model('Grocery Optimization')
prob.Params.LogToConsole = 0

# Set Quantity Variable
x_names = ['Amount from ' + stores['Store_Name'][i] + ' for product ' + str(grocery_data[i]['names'][n]) for i in grocery_data for n in range(len(grocery_data[i]['names']))]
X = prob.addVars(indices, lb = 0, vtype = GRB.INTEGER, name = x_names)

# Set Dummy Variable if store will be visited
U = prob.addVars(store_code, vtype=GRB.BINARY, name = ["Store " + stores['Store_Name'][i] + " is visited" for i in store_code])

# Dummy for type of transport 
t = len(travel_modes)
T = prob.addVars(t, vtype=GRB.BINARY, name = ["Transport method: " + i for i in travel_modes] )

# Create Objective Function
# Calculate cost of shopping list
total_grocery_cost = sum(X[i,n] * grocery_data[i]['prices'][n] for i in store_code for n in range(len(grocery_data[i]['prices'])))

# Calculating cost of commute
'''Average in-city fuel consumption is 10.7L/100 km 
(https://www.nrcan.gc.ca/sites/nrcan/files/oee/pdf/transportation/tools/fuelratings/2021%20Fuel%20Consumption%20Guide.pdf)
So, the average car in Canada consumes 0.000107 L/m. Given it's CAD 1.716 per litre of fuel, this means a meter costs CAD 0.00018361.
Two-way ticket for the subway is CAD 6.5'''

cost = [0.00018361, 0, 6.5, 0]
dis = ['distance_driving', 'distance_walking', 'distance_transit', 'distance_biking']
distance_cost = 0 
for i in grocery_data:
    for j in range(t):
        if dis[j] == 'distance_transit':
            distance_cost += U[i] * cost[j] * T[j]
        else:
            distance_cost += U[i] * grocery_data[i][dis[j]] * cost[j] * T[j]


Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-10


Create the objective function

In [ ]:
#Final Objective Function
prob.setObjective(total_grocery_cost + distance_cost, GRB.MINIMIZE)

Set the constraints

In [20]:
# Constraints
M = 1000000000000000

# Choose only 1 store
for i in store_code: 
    prob.addConstr(sum(X[i,n] for n in range(len(grocery_data[i]['names']))) <= M*U[i], 
                   name = 'Items of Only Visited Store'+str(i)+' Bought Constraint')
prob.addConstr(sum(U[i] for i in grocery_data) == 1, name = 'Visit Only One Store Constraint')

# Method of travel must be chosen
prob.addConstr(sum(T[i] for i in range(len(travel_modes))) == 1, name = 'Selection of Transport Mode Constraint')

# Nutrients

#Choose amount of calories: 
prob.addConstr(sum(X[i,n] * grocery_data[i]['calories'][n] * grocery_data[i]['servings'][n] for i in store_code for n in range(len(grocery_data[i]['calories']))) <= 21000)

# Choose amount of fat: 
prob.addConstr(sum(X[i,n] * grocery_data[i]['fat'][n] * grocery_data[i]['servings'][n] for i in store_code for n in range(len(grocery_data[i]['fat']))) >= 500)

# Choose amount of satfat: 
prob.addConstr(sum(X[i,n] * grocery_data[i]['satfat'][n] * grocery_data[i]['servings'][n] for i in store_code for n in range(len(grocery_data[i]['satfat']))) >= 140)

# Choose amount of transfat: 
prob.addConstr(sum(X[i,n] * grocery_data[i]['trans'][n] * grocery_data[i]['servings'][n] for i in store_code for n in range(len(grocery_data[i]['trans']))) <= 20)

# Choose amount of cholestrol: 
prob.addConstr(sum(X[i,n] * grocery_data[i]['chol'][n] * grocery_data[i]['servings'][n] for i in store_code for n in range(len(grocery_data[i]['chol']))) <= 2200)

# Choose amount of sodium
prob.addConstr(sum(X[i,n] * grocery_data[i]['na'][n] * grocery_data[i]['servings'][n] for i in store_code for n in range(len(grocery_data[i]['na']))) <= 16100)

# Choose amount of carbs 
prob.addConstr(sum(X[i,n] * grocery_data[i]['cho'][n] * grocery_data[i]['servings'][n] for i in store_code for n in range(len(grocery_data[i]['cho']))) >= 2000)

# Choose amount of fibre 
prob.addConstr(sum(X[i,n] * grocery_data[i]['fibre'][n] * grocery_data[i]['servings'][n] for i in store_code for n in range(len(grocery_data[i]['fibre']))) >= 100)

# Choose amount of sugar 
prob.addConstr(sum(X[i,n] * grocery_data[i]['sugar'][n] * grocery_data[i]['servings'][n] for i in store_code for n in range(len(grocery_data[i]['sugar']))) <= 252)

# Choose amount of protein 
prob.addConstr(sum(X[i,n] * grocery_data[i]['pro'][n] * grocery_data[i]['servings'][n] for i in store_code for n in range(len(grocery_data[i]['pro']))) >= 1400)

# Allergies 
for i in grocery_data: 
    for n in range(len(grocery_data[i]['ingredients'])): 
        if 'peanut'.casefold() in str(grocery_data[i]['ingredients'][n]).casefold():
            prob.addConstr(X[i,n] == 0)

# TRA Categories 
# Need to be legumes
prob.addConstr(sum(X[i,n] * grocery_data[i]['K'][n] for i in store_code for n in range(len(grocery_data[i]['K']))) >= 0.05 * sum(X[i,n] for i in store_code for n in range(len(grocery_data[i]['K']))))

# Need to be Meat products or Substitutes
prob.addConstr(sum(X[i,n] * grocery_data[i]['L'][n] for i in store_code for n in range(len(grocery_data[i]['L']))) >= 0.2 * sum(X[i,n] for i in store_code for n in range(len(grocery_data[i]['K']))))

# Need to have eggs
prob.addConstr(sum(X[i,n] * grocery_data[i]['G'][n] for i in store_code for n in range(len(grocery_data[i]['G']))) >= 0.05 * sum(X[i,n] for i in store_code for n in range(len(grocery_data[i]['G']))))

# Need to have vegetables 
prob.addConstr(sum(X[i,n] * grocery_data[i]['V'][n] for i in store_code for n in range(len(grocery_data[i]['V']))) >= 0.1 * sum(X[i,n] for i in store_code for n in range(len(grocery_data[i]['V']))))

# Need to have fruits 
prob.addConstr(sum(X[i,n] * grocery_data[i]['J'][n] for i in store_code for n in range(len(grocery_data[i]['J']))) >= 0.1 * sum(X[i,n] for i in store_code for n in range(len(grocery_data[i]['J']))))

# Bakery Products 
prob.addConstr(sum(X[i,n] * grocery_data[i]['A'][n] for i in store_code for n in range(len(grocery_data[i]['A']))) >= 0.1 * sum(X[i,n] for i in store_code for n in range(len(grocery_data[i]['A']))))

# Beverages
prob.addConstr(sum(X[i,n] * grocery_data[i]['B'][n] for i in store_code for n in range(len(grocery_data[i]['B']))) <= 0.03 * sum(X[i,n] for i in store_code for n in range(len(grocery_data[i]['B']))))

# Grain Products 
prob.addConstr(sum(X[i,n] * grocery_data[i]['C'][n] for i in store_code for n in range(len(grocery_data[i]['B']))) <= 0.1 * sum(X[i,n] for i in store_code for n in range(len(grocery_data[i]['C']))))


# Amount of products 
prob.addConstrs(X[i,n] <= 2 for i in store_code for n in range(len(grocery_data[i]['names'])))

#Time constraint
dur = ['duration_driving', 'duration_walking', 'duration_transit', 'duration_biking']
prob.addConstr(sum(U[i] * T[j] * grocery_data[i][dur[j]] for i in store_code for j in range(len(dur))) <= 2000, name = 'Time Constraint')

# Add constraint for name of product
prob.addConstr(quicksum(X[i,n] for i in store_code for n in range(len(grocery_data[i]['names'])) if "punch".casefold() in str(grocery_data[i]['names'][n]).casefold()) <= 1)
prob.addConstr(quicksum(X[i,n] for i in store_code for n in range(len(grocery_data[i]['names'])) if "noodles".casefold() in str(grocery_data[i]['names'][n]).casefold()) <= 1)
prob.addConstr(quicksum(X[i,n] for i in store_code for n in range(len(grocery_data[i]['names'])) if "soup".casefold() in str(grocery_data[i]['names'][n]).casefold()) <= 1)
prob.addConstr(quicksum(X[i,n] for i in store_code for n in range(len(grocery_data[i]['names'])) if "beans".casefold() in str(grocery_data[i]['names'][n]).casefold()) <= 2)
prob.addConstr(quicksum(X[i,n] for i in store_code for n in range(len(grocery_data[i]['names'])) if "ramen".casefold() in str(grocery_data[i]['names'][n]).casefold()) <= 2)



prob.optimize()

Print the results

In [21]:
print(f'The total cost of this week\'s grocery shopping list is ${prob.ObjVal:.2f}\n')
ss = 0
for i in store_code:
    if U[i].x != 0:
        ss = i
        print('Shop at '+stores['Store_Name'][i])

for i in range(t):
    if T[i].x != 0:
        print('Go to '+stores['Store_Name'][ss]+' by '+travel_modes[i]+'\n')
        
for v in prob.getVars():
    if v.x > 0:
        if (('Store' not in v.varName) & ('Transport' not in v.varName)):
            print(v.varName, "=",v.x)

cal = 0
pro = 0
fat = 0
satfat = 0
trans = 0
carb = 0
fib = 0
sug = 0
sod = 0
chol = 0
vita = 0
vitc = 0
Ca = 0
Fe = 0
K = 0

for v in prob.getVars():
    if (v.x > 0) & (v.index <= len(indices)-1):   
        cal += grocery_data[ss]['calories'][indices[v.index][1]] * v.x * grocery_data[ss]['servings'][indices[v.index][1]]
        pro += grocery_data[ss]['pro'][indices[v.index][1]] * v.x * grocery_data[ss]['servings'][indices[v.index][1]]
        fat += grocery_data[ss]['fat'][indices[v.index][1]] * v.x * grocery_data[ss]['servings'][indices[v.index][1]]
        satfat += grocery_data[ss]['satfat'][indices[v.index][1]] * v.x * grocery_data[ss]['servings'][indices[v.index][1]]
        trans += grocery_data[ss]['trans'][indices[v.index][1]] * v.x * grocery_data[ss]['servings'][indices[v.index][1]]
        carb += grocery_data[ss]['cho'][indices[v.index][1]] * v.x * grocery_data[ss]['servings'][indices[v.index][1]]
        fib += grocery_data[ss]['fibre'][indices[v.index][1]] * v.x * grocery_data[ss]['servings'][indices[v.index][1]]
        sug += grocery_data[ss]['sugar'][indices[v.index][1]] * v.x * grocery_data[ss]['servings'][indices[v.index][1]]
        sod += grocery_data[ss]['na'][indices[v.index][1]] * v.x * grocery_data[ss]['servings'][indices[v.index][1]]
        chol += grocery_data[ss]['chol'][indices[v.index][1]] * v.x * grocery_data[ss]['servings'][indices[v.index][1]]
        vita += grocery_data[ss]['vita'][indices[v.index][1]] * v.x * grocery_data[ss]['servings'][indices[v.index][1]]
        vitc += grocery_data[ss]['vitc'][indices[v.index][1]] * v.x * grocery_data[ss]['servings'][indices[v.index][1]]
        Ca += grocery_data[ss]['calcium'][indices[v.index][1]] * v.x * grocery_data[ss]['servings'][indices[v.index][1]]
        Fe += grocery_data[ss]['iron'][indices[v.index][1]] * v.x * grocery_data[ss]['servings'][indices[v.index][1]]
        K += grocery_data[ss]['k'][indices[v.index][1]] * v.x * grocery_data[ss]['servings'][indices[v.index][1]]
              
print('\nTotal Calories (kcal) =',cal,
      '\nTotal Protein (g) =',pro,
      '\nTotal Fat (g) =',fat,
      '\nTotal Saturated Fat (g) =',satfat,
      '\nTotal Trans Fat (g) =',trans,
      '\nTotal Carbs (g) =',carb,
      '\nTotal Fibres (g) =',fib,
      '\nTotal Sugar (g) =',sug,
      '\nTotal Sodium (mg) =',sod,
      '\nTotal Cholestrol (mg) =',chol,
      '\nTotal Vitamin A (mg) =',vita,
      '\nTotal Vitamin C (mg) =',vitc,
      '\nTotal Calcium (mg) =',Ca,
      '\nTotal Iron (mg) =',Fe,
      '\nTotal Potassium (mg) =',K)

The total cost of this week's grocery shopping list is $28.36

Shop at No Frills Joe's
Go to No Frills Joe's by Biking

Amount from No Frills Joe's for product Chocolate Wafers = 1.0
Amount from No Frills Joe's for product Apple J Sparkling Apple Drink = 1.0
Amount from No Frills Joe's for product Sparkling Passion Fruit Beverage = 1.0
Amount from No Frills Joe's for product Parle Krackjack Cookies = 1.0
Amount from No Frills Joe's for product Grade A White Eggs, Large = 1.0
Amount from No Frills Joe's for product Whole Leaf Spinach = 1.0
Amount from No Frills Joe's for product Chicken Hot Dogs = 2.0
Amount from No Frills Joe's for product Luncheon Meat = 1.0
Amount from No Frills Joe's for product Whole Flaxseed = 1.0
Amount from No Frills Joe's for product Sliced Mushrooms = 1.0
Amount from No Frills Joe's for product Yellow Split Peas = 2.0
Amount from No Frills Joe's for product Lupini Beans = 2.0

Total Calories (kcal) = 20991.219504643963 
Total Protein (g) = 1401.1034834144184 
